In [1]:
# do a hacky import since brainaudio requires python 3.12 and the .wfst env requires an older
# python version 
import importlib                                                                                                                                                                                                    
import wfst_utils
importlib.reload(wfst_utils)
from wfst_utils import build_lm_decoder, lm_decode, arrange_logits, _cer_and_wer, augment_nbest
import sys
import numpy as np
import re
import pickle
import pandas as pd
import sys                                                                                                                                                                                                          

In [2]:
dataset = 'b2t_24'
load_lm = True

In [10]:
if dataset == "b2t_25":
    blank_penalty = np.log(90)
    acoustic_scale = 0.325
    beam = 17
    rescore = True
    base_path = "/home/ebrahim/data2/brain2text/b2t_25/logits/baseline_rnn_ucd_seeds_1_to_9/"
    save_path = "/home/ebrahim/data2/brain2text/b2t_25/wfst_outputs/"

    
if dataset == "b2t_24":
    blank_penalty = np.log(7)
    acoustic_scale = 0.5
    beam = 18
    rescore = True
    base_path = "/home/ebrahim/data2/brain2text/b2t_24/logits/"
    save_path = "/home/ebrahim/data2/brain2text/b2t_24/wfst_outputs/"
    seeds_list = [0]
    
lmDir = "/home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test"
nbest_value = 100
return_n_best = True

In [5]:
if load_lm and 'ngramDecoder' not in globals(): 
    
    ngramDecoder = build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest_value,
        beam=beam,
    )
    print("loaded LM")

loaded LM


I0212 01:42:58.905295 1551219 brain_speech_decoder.h:52] Reading fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/TLG.fst
I0212 01:45:01.212980 1551219 brain_speech_decoder.h:58] Reading lm fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/G.fst
I0212 01:45:27.788199 1551219 brain_speech_decoder.h:70] Reading rescore fst /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/G_no_prune.fst
I0212 01:54:10.200368 1551219 brain_speech_decoder.h:81] Reading symbol table /home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test/words.txt


In [26]:
# folder name
folder_name = ["time_masked_transformer_seed_1"]
logits_file_name = ["logits_val.npz"]
nbest_save_path_arr = ["time_masked_transformer/nbest_wfst_seed_1_neurips"]


In [27]:
rescore_bool = True

for seed, fn, lm, nb in zip(seeds_list, folder_name, logits_file_name, nbest_save_path_arr):

    logits_all = np.load(f"{base_path}{fn}/{lm}")

    nbest_save_path = f"{save_path}{nb}.pkl"

    if dataset == "b2t_25":
        
        nbest_augmented_save_path = f"{save_path}{nb}_augmented"

    nbest_outputs = []
    nbest_augmented_outputs = []

    for trial_idx in range(len(logits_all.keys())):
        
        if trial_idx % 100 == 0:
            
            print(f"Decoding trial {trial_idx}...")
        
        logits = logits_all[f'arr_{trial_idx}']
            
        rearranged_logits = arrange_logits(logits)
        
        nbest = lm_decode(
                        ngramDecoder,
                        rearranged_logits[0],
                        blankPenalty=blank_penalty,
                        returnNBest=return_n_best,
                        rescore=rescore_bool,
                    )

        if dataset == 'b2t_25':
            
            nbest_augmented = augment_nbest(nbest, acoustic_scale=acoustic_scale, top_candidates_to_augment=20, score_penalty_percent=0.01)
            nbest_augmented_outputs.append(nbest_augmented)
            
            
        nbest_outputs.append(nbest)
        
        with open(nbest_save_path, 'wb') as f:
            pickle.dump(nbest_outputs, f)
            
        if dataset == 'b2t_25':
            with open(nbest_augmented_save_path, 'wb') as f:
                pickle.dump(nbest_augmented_outputs, f)

Decoding trial 0...
Decoding trial 100...
Decoding trial 200...
Decoding trial 300...
Decoding trial 400...
Decoding trial 500...
Decoding trial 600...
Decoding trial 700...
Decoding trial 800...


In [14]:
nbest_save_path

'/home/ebrahim/data2/brain2text/b2t_24/wfst_outputs//.pkl'